In [32]:
import pandas as pd
import glob
import os
import requests
import pprint

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [33]:
all_files = glob.glob(r'../data/*.csv')

In [34]:
# df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
all_data = []
for f in all_files:
    data = pd.read_csv(f)
    collection_name = os.path.splitext(os.path.basename(f))[0].capitalize()
    data['collection'] = collection_name
    all_data.append(data)

df = pd.concat(all_data, ignore_index = True)

In [35]:
df.head()

,item_type,title,creators,first_name,last_name,ean_isbn13,upc_isbn10,description,publisher,publish_date,group,tags,notes,price,length,number_of_discs,number_of_players,age_group,ensemble,aspect_ratio,esrb,rating,review,review_date,status,began,completed,added,copies,collection
0,book,Lifelines,Leana Wen,Leana,Wen,9781250186232,1250186234,"From medical expert Leana Wen, MD,Lifelines is...",Henry Holt &amp; Company,2021-07-27,NaN,NaN,NaN,NaN,352.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-13,1,Norfolk
1,book,Edith's Story: The True Story of a Young Girl'...,Edith Velmans,Edith,Velmans,9780553381108,0553381105,"In 1940, while the Germans occupied Holland, f...",Bantam,2001-01-02,NaN,NaN,NaN,NaN,256.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-13,1,Norfolk
2,book,The Flight Attendant (Television Tie-In Edition),Chris Bohjalian,Chris,Bohjalian,9780593314005,059331400X,Now a limited series on HBO Max starring Kaley...,Knopf Doubleday Publishing Group,2020-12-08,NaN,NaN,NaN,NaN,368.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-13,1,Norfolk
3,book,The Dalai Lama's Cat,David Michie,David,Michie,9781401940584,1401940587,Oh! How adorable! I didn&#039;t know you had a...,Hay House Visions,2012-10-01,NaN,NaN,NaN,NaN,240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-13,1,Maple
4,book,Joey Pigza Swallowed the Key (Joey Pigza Books),Jack Gantos,Jack,Gantos,9780064408332,64408337,Joey Pigza can&#039;t sit still. He can&#039;t...,HarperCollins,2000-04-30,NaN,NaN,NaN,NaN,160.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-13,1,Maple


In [47]:
# Now loop through and call APIs for each entry
df['categories'] = None
for index, row in df.head(10).iterrows():
    
    print(f"-- Getting info for {row['title']}")
    book_isbn = row["ean_isbn13"]

    # Pull subjects data from Open Library API
    response = requests.get(f"https://openlibrary.org/isbn/{book_isbn}.json")
    open_lib_details = response.json()

    if 'subjects' in open_lib_details:
        subjects = open_lib_details['subjects']
        cleaned_subjects = ', '.join([subj.replace(',', '') for subj in subjects])
        df.at[index, 'subjects'] = cleaned_subjects

    if 'physical_format' in open_lib_details:
        df.at[index, 'physical_format'] = open_lib_details['physical_format']

    # Pull categories data from Google Books API
    response = requests.get(f"https://www.googleapis.com/books/v1/volumes?q=isbn:{book_isbn}")
    google_details = response.json()
    pprint.pprint(google_details)

    try:
        volume_info = google_details['items'][0]['volumeInfo']

        if 'categories' in volume_info:
            # Remove commas now, so I can accurately split categories on comma post-join
            cleaned_categories = ', '.join([cat.replace(',', '') for cat in volume_info['categories']])
            df.at[index, 'categories'] = cleaned_categories
            
    except:
        print(f"No Google details found for ISBN {book_isbn}")
        df.at[index, 'categories'] = None
    

-- Getting info for Lifelines
{'items': [{'accessInfo': {'accessViewStatus': 'NONE',
                           'country': 'US',
                           'embeddable': False,
                           'epub': {'isAvailable': False},
                           'pdf': {'isAvailable': False},
                           'publicDomain': False,
                           'quoteSharingAllowed': False,
                           'textToSpeechPermission': 'ALLOWED',
                           'viewability': 'NO_PAGES',
                           'webReaderLink': 'http://play.google.com/books/reader?id=lIS-zQEACAAJ&hl=&source=gbs_api'},
            'etag': '93qxjc5SwDY',
            'id': 'lIS-zQEACAAJ',
            'kind': 'books#volume',
            'saleInfo': {'country': 'US',
                         'isEbook': False,
                         'saleability': 'NOT_FOR_SALE'},
            'searchInfo': {'textSnippet': 'Here, in gripping detail, Wen lays '
                                   

In [41]:
df.head(10)

,item_type,title,creators,first_name,last_name,ean_isbn13,upc_isbn10,description,publisher,publish_date,group,tags,notes,price,length,number_of_discs,number_of_players,age_group,ensemble,aspect_ratio,esrb,rating,review,review_date,status,began,completed,added,copies,collection,categories,testing
0,book,Lifelines,Leana Wen,Leana,Wen,9781250186232,1250186234,"From medical expert Leana Wen, MD,Lifelines is...",Henry Holt &amp; Company,2021-07-27,NaN,NaN,NaN,NaN,352.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-13,1,Norfolk,Social Science,NaN
1,book,Edith's Story: The True Story of a Young Girl'...,Edith Velmans,Edith,Velmans,9780553381108,0553381105,"In 1940, while the Germans occupied Holland, f...",Bantam,2001-01-02,NaN,NaN,NaN,NaN,256.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-13,1,Norfolk,Biography & Autobiography,NaN
2,book,The Flight Attendant (Television Tie-In Edition),Chris Bohjalian,Chris,Bohjalian,9780593314005,059331400X,Now a limited series on HBO Max starring Kaley...,Knopf Doubleday Publishing Group,2020-12-08,NaN,NaN,NaN,NaN,368.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-13,1,Norfolk,Fiction,NaN
3,book,The Dalai Lama's Cat,David Michie,David,Michie,9781401940584,1401940587,Oh! How adorable! I didn&#039;t know you had a...,Hay House Visions,2012-10-01,NaN,NaN,NaN,NaN,240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-13,1,Maple,Fiction,NaN
4,book,Joey Pigza Swallowed the Key (Joey Pigza Books),Jack Gantos,Jack,Gantos,9780064408332,64408337,Joey Pigza can&#039;t sit still. He can&#039;t...,HarperCollins,2000-04-30,NaN,NaN,NaN,NaN,160.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-13,1,Maple,Juvenile Fiction,NaN
5,book,"Are You There God? It's Me, Margaret",Judy Blume,Judy,Blume,9780440404194,440404193,"Are You There God? It&#039;s Me, Margaret No o...",Yearling,1972-01-01,NaN,NaN,NaN,NaN,160.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-13,1,Maple,Juvenile Fiction,NaN
6,book,"Hannah Arendt: For Love of the World, Second E...",Elisabeth Young-Bruehl,Elisabeth,Young-Bruehl,9780300105889,300105886,"This highly acclaimed, prize-winning biography...",Yale University Press,2004-10-11,NaN,NaN,NaN,NaN,620.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-13,1,Maple,Biography & Autobiography,NaN
7,book,Heroes and Saints and Other Plays: Giving Up t...,Cherrie Moraga,Cherrie,Moraga,9780931122743,931122740,Heroes and Saints &amp; Other Plays is Chicana...,West End Press,1994-12-31,NaN,NaN,NaN,NaN,149.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-13,1,Maple,Drama,NaN
8,book,Ceremony (Contemporary American Fiction Series),Leslie Marmon Silko,Leslie,Silko,9780140086836,140086838,"Tayo, a young Native American, has been a pris...",Penguin Books,1986-03-04,NaN,NaN,NaN,NaN,260.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-13,1,Maple,Computers,NaN
9,book,And the Bridge Is Love,Faye Moskowitz,Faye,Moskowitz,9780807063279,807063274,NaN,Beacon Press,1993-08-01,NaN,NaN,NaN,NaN,160.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-13,1,Maple,Biography & Autobiography,NaN
